In [1]:
import pandas as pd
from scripts import project_functions

# Load data from National Names
national_df = project_functions.load_and_process_one('../../data/raw/national/NationalNames.csv')
# Load 4 parts of raw data on State Names
state_df = project_functions.load_and_process_many("../../data/raw/state")


Importing scripts.project_functions


In [2]:
national_df

,Id,Name,Year,Gender,Count
0,1,Mary,1880,F,7065
1,2,Anna,1880,F,2604
2,3,Emma,1880,F,2003
3,4,Elizabeth,1880,F,1939
4,5,Minnie,1880,F,1746
...,...,...,...,...,...
1825428,1825429,Zykeem,2014,M,5
1825429,1825430,Zymeer,2014,M,5
1825430,1825431,Zymiere,2014,M,5
1825431,1825432,Zyran,2014,M,5


In [3]:
state_df

,Id,Name,Year,Gender,State,Count
0,1,Mary,1910,F,AK,14
1,2,Annie,1910,F,AK,12
2,3,Anna,1910,F,AK,10
3,4,Margaret,1910,F,AK,8
4,5,Helen,1910,F,AK,7
...,...,...,...,...,...,...
5647421,5647422,Seth,2014,M,WY,5
5647422,5647423,Spencer,2014,M,WY,5
5647423,5647424,Tyce,2014,M,WY,5
5647424,5647425,Victor,2014,M,WY,5
